**House Prices - Feature Selection**

Importo las clases a utilizar.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import warnings
warnings.filterwarnings("ignore")

Se cargan los dataframes de train y test a los que ya se les ha manejado los valores faltantes, escalado los atributos y codificado las variables categóricas.

In [ ]:
pd.pandas.set_option('display.max_columns', None)

In [ ]:
df_train = pd.read_csv('escalado_train.csv')
df_train.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1,12.247694,0.235294,0.75,0.418208,0.366344,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.036765,0.098361,0.0,0.0,1.0,1.0,0.666667,0.12250,0.666667,1.0,1.00,0.75,0.75,0.25,1.000000,0.125089,0.833333,0.0,0.064212,0.140098,1.0,1.00,1.0,1.0,0.356155,0.413559,0.0,0.577712,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,1.0,0.000000,0.2,0.8,0.046729,0.666667,0.50,0.386460,0.666667,1.0,1.0,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.50,0.666667,0.75,0.0,0.0,0.0
1,2,12.109011,0.000000,0.75,0.495064,0.391317,1.0,1.0,0.000000,0.333333,1.0,0.50,0.0,0.500000,0.2,1.0,0.75,0.6,0.555556,0.875,0.227941,0.524590,0.0,0.0,0.4,0.3,0.333333,0.00000,0.333333,1.0,0.50,0.75,0.75,1.00,0.666667,0.173281,0.833333,0.0,0.121575,0.206547,1.0,1.00,1.0,1.0,0.503056,0.000000,0.0,0.470245,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.333333,0.333333,1.0,0.333333,0.6,0.8,0.289720,0.666667,0.50,0.324401,0.666667,1.0,1.0,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.363636,0.25,0.666667,0.75,0.0,0.0,0.0
2,3,12.317167,0.235294,0.75,0.434909,0.422359,1.0,1.0,0.333333,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.051471,0.114754,0.0,0.0,1.0,1.0,0.666667,0.10125,0.666667,1.0,1.00,0.75,0.75,0.50,1.000000,0.086109,0.833333,0.0,0.185788,0.150573,1.0,1.00,1.0,1.0,0.383441,0.419370,0.0,0.593095,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,1.0,0.333333,0.6,0.8,0.065421,0.666667,0.50,0.428773,0.666667,1.0,1.0,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.727273,0.50,0.666667,0.75,0.0,0.0,0.0
3,4,11.849398,0.294118,0.75,0.388581,0.390295,1.0,1.0,0.333333,0.333333,1.0,0.25,0.0,0.727273,0.4,1.0,0.75,1.0,0.666667,0.500,0.669118,0.606557,0.0,0.0,0.2,0.4,0.333333,0.00000,0.333333,1.0,0.25,0.50,1.00,0.25,0.666667,0.038271,0.833333,0.0,0.231164,0.123732,1.0,0.75,1.0,1.0,0.399941,0.366102,0.0,0.579157,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,1.0,0.333333,0.8,0.4,0.074766,0.333333,0.75,0.452750,0.666667,1.0,1.0,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.00,0.666667,0.00,0.0,0.0,0.0
4,5,12.429216,0.235294,0.75,0.513123,0.468761,1.0,1.0,0.333333,0.333333,1.0,0.50,0.0,1.000000,0.4,1.0,0.75,1.0,0.777778,0.500,0.058824,0.147541,0.0,0.0,1.0,1.0,0.666667,0.21875,0.666667,1.0,1.00,0.75,0.75,0.75,1.000000,0.116052,0.833333,0.0,0.209760,0.187398,1.0,1.00,1.0,1.0,0.466237,0.509927,0.0,0.666523,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,1.0,0.333333,0.6,0.8,0.074766,0.666667,0.75,0.589563,0.666667,1.0,1.0,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000,0.50,0.666667,0.75,0.0,0.0,0.0


In [ ]:
df_test = pd.read_csv('escalado_test.csv')
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,1461,0.000000,0.50,0.495064,0.428726,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.363636,0.2,1.0,0.75,0.6,0.444444,0.625,0.360294,0.819672,0.0,0.0,1.0,1.0,0.333333,0.0000,0.333333,1.0,0.5,0.50,0.75,0.25,0.166667,0.082920,0.500000,0.097693,0.115582,0.144354,1.0,0.50,1.0,1.0,0.373438,0.000000,0.0,0.349081,0.0,0.0,0.333333,0.0,0.250,0.333333,0.333333,0.250000,1.0,0.000000,0.2,0.8,0.457944,0.333333,0.25,0.514810,0.666667,1.0,1.0,0.163361,0.000000,0.0,0.0,0.25,0.0,0.0,0.5,1.0,0.000000,0.454545,1.0,0.666667,0.75,0.0,0.0,0.0
1,1462,0.000000,0.75,0.499662,0.468857,1.0,1.0,0.333333,0.333333,1.0,0.25,0.0,0.363636,0.4,1.0,0.75,0.6,0.555556,0.625,0.382353,0.868852,1.0,0.0,0.2,0.1,0.666667,0.0675,0.333333,1.0,0.5,0.50,0.75,0.25,0.666667,0.163536,0.833333,0.000000,0.173801,0.217512,1.0,0.50,1.0,1.0,0.522632,0.000000,0.0,0.488544,0.0,0.0,0.333333,0.5,0.375,0.333333,0.666667,0.333333,1.0,0.000000,0.2,0.8,0.485981,0.333333,0.25,0.220028,0.666667,1.0,1.0,0.458576,0.065814,0.0,0.0,0.00,0.0,0.0,1.0,0.0,0.806452,0.454545,1.0,0.666667,0.75,0.0,0.0,0.0
2,1463,0.235294,0.75,0.466207,0.462769,1.0,1.0,0.333333,0.333333,1.0,0.00,0.0,0.590909,0.4,1.0,0.75,1.0,0.444444,0.500,0.095588,0.213115,0.0,0.0,1.0,1.0,0.333333,0.0000,0.333333,1.0,1.0,0.75,0.75,0.25,1.000000,0.140149,0.833333,0.000000,0.058647,0.151882,1.0,0.75,1.0,1.0,0.386718,0.339467,0.0,0.560546,0.0,0.0,0.666667,0.5,0.375,0.333333,0.333333,0.333333,1.0,0.333333,0.6,0.8,0.121495,1.000000,0.50,0.339915,0.666667,1.0,1.0,0.247375,0.062157,0.0,0.0,0.00,0.0,0.0,0.5,1.0,0.000000,0.181818,1.0,0.666667,0.75,0.0,0.0,0.0
3,1464,0.235294,0.75,0.485693,0.398875,1.0,1.0,0.333333,0.333333,1.0,0.00,0.0,0.590909,0.4,1.0,0.75,1.0,0.555556,0.625,0.088235,0.213115,0.0,0.0,1.0,1.0,0.666667,0.0125,0.333333,1.0,1.0,0.50,0.75,0.25,1.000000,0.106662,0.833333,0.000000,0.138699,0.151555,1.0,1.00,1.0,1.0,0.385901,0.328329,0.0,0.555075,0.0,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.416667,1.0,0.333333,0.8,0.8,0.112150,1.000000,0.50,0.331453,0.666667,1.0,1.0,0.420070,0.065814,0.0,0.0,0.00,0.0,0.0,1.0,1.0,0.000000,0.454545,1.0,0.666667,0.75,0.0,0.0,0.0
4,1465,0.588235,0.75,0.265271,0.263841,1.0,1.0,0.333333,1.000000,1.0,0.00,0.0,0.909091,0.4,1.0,1.00,0.6,0.777778,0.500,0.132353,0.311475,0.0,0.0,0.6,0.5,0.333333,0.0000,0.666667,1.0,1.0,0.75,0.75,0.25,0.666667,0.046598,0.833333,0.000000,0.435360,0.209493,1.0,1.00,1.0,1.0,0.508416,0.000000,0.0,0.475254,0.0,0.0,0.666667,0.0,0.250,0.333333,0.666667,0.250000,1.0,0.000000,0.2,0.8,0.168224,0.666667,0.50,0.356841,0.666667,1.0,1.0,0.000000,0.149909,0.0,0.0,0.30,0.0,0.0,1.0,1.0,0.000000,0.000000,1.0,0.666667,0.75,0.0,0.0,0.0


Se extraen las variables independientes 'X' y la variable objetivo 'y' del DataFrame de train, mientras que del conjunto de test se extraen únicamente las variables independientes 'X', preparándolos para el proceso de modelado.

In [ ]:
X_train = df_train.drop(['Id','SalePrice'],axis='columns')
X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_nan,MasVnrArea_nan,GarageYrBlt_nan
0,0.235294,0.75,0.418208,0.366344,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.036765,0.098361,0.0,0.0,1.0,1.0,0.666667,0.12250,0.666667,1.0,1.00,0.75,0.75,0.25,1.000000,0.125089,0.833333,0.0,0.064212,0.140098,1.0,1.00,1.0,1.0,0.356155,0.413559,0.0,0.577712,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.500000,1.0,0.000000,0.2,0.8,0.046729,0.666667,0.50,0.386460,0.666667,1.0,1.0,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.50,0.666667,0.75,0.0,0.0,0.0
1,0.000000,0.75,0.495064,0.391317,1.0,1.0,0.000000,0.333333,1.0,0.50,0.0,0.500000,0.2,1.0,0.75,0.6,0.555556,0.875,0.227941,0.524590,0.0,0.0,0.4,0.3,0.333333,0.00000,0.333333,1.0,0.50,0.75,0.75,1.00,0.666667,0.173281,0.833333,0.0,0.121575,0.206547,1.0,1.00,1.0,1.0,0.503056,0.000000,0.0,0.470245,0.000000,0.5,0.666667,0.0,0.375,0.333333,0.333333,0.333333,1.0,0.333333,0.6,0.8,0.289720,0.666667,0.50,0.324401,0.666667,1.0,1.0,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.363636,0.25,0.666667,0.75,0.0,0.0,0.0
2,0.235294,0.75,0.434909,0.422359,1.0,1.0,0.333333,0.333333,1.0,0.00,0.0,0.636364,0.4,1.0,0.75,1.0,0.666667,0.500,0.051471,0.114754,0.0,0.0,1.0,1.0,0.666667,0.10125,0.666667,1.0,1.00,0.75,0.75,0.50,1.000000,0.086109,0.833333,0.0,0.185788,0.150573,1.0,1.00,1.0,1.0,0.383441,0.419370,0.0,0.593095,0.333333,0.0,0.666667,0.5,0.375,0.333333,0.666667,0.333333,1.0,0.333333,0.6,0.8,0.065421,0.666667,0.50,0.428773,0.666667,1.0,1.0,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.727273,0.50,0.666667,0.75,0.0,0.0,0.0
3,0.294118,0.75,0.388581,0.390295,1.0,1.0,0.333333,0.333333,1.0,0.25,0.0,0.727273,0.4,1.0,0.75,1.0,0.666667,0.500,0.669118,0.606557,0.0,0.0,0.2,0.4,0.333333,0.00000,0.333333,1.0,0.25,0.50,1.00,0.25,0.666667,0.038271,0.833333,0.0,0.231164,0.123732,1.0,0.75,1.0,1.0,0.399941,0.366102,0.0,0.579157,0.333333,0.0,0.333333,0.0,0.375,0.333333,0.666667,0.416667,1.0,0.333333,0.8,0.4,0.074766,0.333333,0.75,0.452750,0.666667,1.0,1.0,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.090909,0.00,0.666667,0.00,0.0,0.0,0.0
4,0.235294,0.75,0.513123,0.468761,1.0,1.0,0.333333,0.333333,1.0,0.50,0.0,1.000000,0.4,1.0,0.75,1.0,0.777778,0.500,0.058824,0.147541,0.0,0.0,1.0,1.0,0.666667,0.21875,0.666667,1.0,1.00,0.75,0.75,0.75,1.000000,0.116052,0.833333,0.0,0.209760,0.187398,1.0,1.00,1.0,1.0,0.466237,0.509927,0.0,0.666523,0.333333,0.0,0.666667,0.5,0.500,0.333333,0.666667,0.583333,1.0,0.333333,0.6,0.8,0.074766,0.666667,0.75,0.589563,0.666667,1.0,1.0,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.000000,0.50,0.666667,0.75,0.0,0.0,0.0


In [ ]:
y_train = df_train.SalePrice
y_train.head()

,SalePrice
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216


Se elimina la columna 'Id' de X_test porque no aporta información relevante para el modelo, ya que es únicamente un identificador único para cada registro y no tiene relación con las características predictivas. Mantener esta columna podría introducir ruido en el entrenamiento o evaluación del modelo.

In [ ]:
X_test = df_test.drop(['Id'], axis='columns')

Se utiliza Lasso Regression para seleccionar atributos aplicando regularización L1, que reduce los coeficientes menos relevantes a cero. Con SelectFromModel, se identifican las características con coeficientes no nulos. El parámetro alpha controla la penalización, donde un valor mayor selecciona menos atributos. En este caso, se establece alpha=0.005 para equilibrar la selección.

In [ ]:
modelo_seleccion_atributos = SelectFromModel(Lasso(alpha=0.005, random_state=2003))
modelo_seleccion_atributos.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.005, random_state=2003))

In [ ]:
modelo_seleccion_atributos.get_support()

array([ True,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False,  True, False,
       False,  True,  True, False, False, False, False, False, False,
       False, False,  True, False,  True, False, False, False, False,
       False, False, False,  True,  True, False,  True, False, False,
        True,  True, False, False, False, False, False,  True, False,
       False,  True,  True,  True, False,  True,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False])

Aquí se puede observar cuántos atributos fueron seleccionados por el modelo, cuántos quedaron con coeficientes diferentes de cero y cuántos se redujeron a cero debido a la regularización L1 de Lasso. Esto permite evaluar el impacto de la selección de atributos sobre el conjunto inicial.

In [ ]:
atributos_seleccionados = X_train.columns[(modelo_seleccion_atributos.get_support())]
print(f'total atributos: {X_train.shape[1]}')
print('atributos seleccionados: {}'.format(len(atributos_seleccionados)))
print('atributos con coeficientes disminuidos a cero: {}'.format(
    np.sum(modelo_seleccion_atributos.estimator_.coef_ == 0)))

total atributos: 82
atributos seleccionados: 21
atributos con coeficientes disminuidos a cero: 61


In [ ]:
atributos_seleccionados

Index(['MSSubClass', 'MSZoning', 'Neighborhood', 'OverallQual', 'YearRemodAdd',
       'RoofStyle', 'BsmtQual', 'BsmtExposure', 'HeatingQC', 'CentralAir',
       '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'KitchenQual', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'PavedDrive',
       'SaleCondition'],
      dtype='object')

In [ ]:
X_train = X_train[atributos_seleccionados]
X_train.shape

(1460, 21)

In [ ]:
X_test = X_test[atributos_seleccionados]
X_test.shape

(1459, 21)

Se dividen los datos en train y test mediante train_test_split para evaluar el rendimiento del modelo en datos no vistos, garantizando una validación más robusta y evitando el sobreajuste.

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_train, y_train,test_size=0.2, random_state=2003)

In [ ]:
x_train.shape, x_test.shape

((1168, 21), (292, 21))

Se establecen ciertos parámetros a ser aplicados para probar cuales son los que mejor funcionan.

In [ ]:
parametros = {
    'n_estimators': [600, 1000, 1250, 1500],
    'max_depth': [6, 7, 8, 9],
    'learning_rate': [0.01, 0.015, 0.02, 0.025, 0.03],
    'subsample': [0, 0.1, 0.2, 0.35, 0.5]
}



Se realiza una búsqueda de hiperparámetros con GridSearchCV para optimizar el modelo GradientBoostingRegressor. Se define un conjunto de parámetros (param_grid) y se utiliza validación cruzada con 5 particiones (cv=5). La métrica de evaluación es el error cuadrático medio negativo (neg_mean_squared_error). Tras ajustar el modelo con los datos de entrenamiento, se obtienen los mejores parámetros y el puntaje mínimo de error cuadrático medio (MSE).

In [ ]:
modelo = GradientBoostingRegressor(random_state=2003)

In [ ]:
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=parametros,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

print("Buscando los mejores parámetros para GradientBoostingRegressor...")
grid_search.fit(x_train, y_train)

mejores_parametros = grid_search.best_params_
mejor_puntaje = -grid_search.best_score_
print(f"Mejores parámetros: {mejores_parametros}")
print(f"Mejor puntaje (MSE): {mejor_puntaje}")


Buscando los mejores parámetros para GradientBoostingRegressor...
Mejores parámetros: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 1000, 'subsample': 0.1}
Mejor puntaje (MSE): 0.016842766533032363


Finalmente, se evalúa el modelo óptimo en el conjunto de prueba, mostrando las primeras predicciones generadas.

In [ ]:
mejor_modelo = grid_search.best_estimator_
predicciones = mejor_modelo.predict(x_test)
print("Primeras predicciones en el conjunto de prueba:")
print(predicciones[:10])

Primeras predicciones en el conjunto de prueba:
[12.19398635 11.71595042 11.57446372 12.17806045 12.11570351 11.97483679
 12.05928181 11.6250591  11.87708667 11.8579355 ]


Se evalúa el rendimiento del modelo en el conjunto de prueba calculando el error cuadrático medio (MSE) y el coeficiente de determinación (R²). El MSE mide la magnitud promedio de los errores en las predicciones, mientras que R² evalúa qué tan bien el modelo explica la variabilidad de los datos, siendo 1 el valor óptimo.

In [ ]:
y_pred = mejor_modelo.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print(f"MSE en test: {mse:.4f}")

r2 = mejor_modelo.score(x_test, y_test)
print(f"R^2 en test: {r2:.4f}")

MSE en test: 0.0175
R^2 en test: 0.8793


Una vez entrenado el mejor modelo, se predicen los precios para los registros del DataFrame de test, generando las estimaciones finales basadas en las características seleccionadas.

In [ ]:
predicciones = mejor_modelo.predict(X_test)

Se crea un DataFrame concatenando la columna 'Id' con las predicciones, las cuales se transforman exponencialmente para revertir la conversión logarítmica realizada previamente, recuperando así los valores originales de las predicciones.

In [ ]:
entrega = pd.DataFrame({'Id': df_test.Id,'SalePrice': np.exp(predicciones)})

In [ ]:
entrega.head()

,Id,SalePrice
0,1461,114584.991064
1,1462,132832.896234
2,1463,169716.753171
3,1464,179681.983644
4,1465,190538.387553


Se convierte el DataFrame a 'csv' y con eso estaría listo para ser entregado en kaggle.

In [ ]:
entrega.to_csv('entrega.csv',index=False)

Finalmente, para evitar tener que entrenar nuevamente el modelo, se pueden guardar tanto el modelo entrenado como los mejores parámetros. Esto permite cargarlos posteriormente para usarlos o compartirlos sin necesidad de repetir el proceso de entrenamiento.

In [ ]:
with open('mejor_modelo_gbr.pkl', 'wb') as file:
    pickle.dump(grid_search, file)

with open('mejores_parametros_gbr.txt', 'w') as file:
    file.write(f"Mejores parámetros: {mejores_parametros}\n")
    file.write(f"Mejor puntaje (MSE): {mejor_puntaje}")


In [ ]:
with open('mejor_modelo_gbr.pkl', 'rb') as file:
    grid_search_cargado = pickle.load(file)

mejor_modelo_cargado = grid_search_cargado.best_estimator_
predicciones = mejor_modelo_cargado.predict(x_test)

with open('mejores_parametros_gbr.txt', 'r') as file:
    parametros_cargados = file.read()

print(parametros_cargados)

Mejores parámetros: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 1000, 'subsample': 0.1}
Mejor puntaje (MSE): 0.016842766533032363


**Conclusión**


Este proyecto me permitió combinar y aplicar los conocimientos adquiridos en machine learning, integrando cada etapa clave como el análisis exploratorio de datos (EDA), el feature engineering y la selección de atributos para construir un modelo sólido. Entendí cómo cada decisión, desde el manejo de valores faltantes hasta la elección de atributos relevantes, influye directamente en el rendimiento del modelo. Además, el proceso mostró la importancia de iterar constantemente, ajustando y evaluando las decisiones para mejorar los resultados. Esta experiencia refuerza cómo un enfoque estructurado y reflexivo puede convertir un problema complejo en una solución eficiente y bien fundamentada.






